## **Preparations**

In [1]:
import os
import sys
import os.path as op
import numpy as np
from functools import partial

import torch

sys.path.append("..")
from mtecg.classifier import ECGClassifier
from mtecg.evaluation import evaluate_from_dataframe
from mtecg.utils import load_ecg_dataframe


SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.cuda.manual_seed(SEED)

c:\Anaconda3\envs\ecg\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda3\envs\ecg\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: llow is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
singletask_scar_model_path = "../../ecg/ecg-cnn-local/trained_models_02112022/single-task-scar/resnet34d_384_LVEF50"
singletask_lvef_model_path = "../../ecg/ecg-cnn-local/trained_models_02112022/single-task-lvef/resnet34d_384_LVEF50"
multitask_model_path = "../../ecg/ecg-cnn-local/trained_models_02112022/1-stage-multitask/resnet34d_384_LVEF50"
multitask_clinical_model_path = "../../ecg/ecg-cnn-local/trained_models_02112022/1-stage-multitask-clinical/resnet34d_384_LVEF50_lstm_1layers_128dims"


singletask_scar_classifier = ECGClassifier(singletask_scar_model_path, model_class="single-task", device="cuda", task="scar")
singletask_lvef_classifier = ECGClassifier(singletask_lvef_model_path, model_class="single-task", device="cuda", task="lvef")
multitask_classifier = ECGClassifier(multitask_model_path, model_class="multi-task", device="cuda",)
multitask_clinical_classifier = ECGClassifier(multitask_clinical_model_path, model_class="multi-task-clinical", device="cuda",)

Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v1.9.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file c:\Users\namea\Desktop\ecg\ecg-cnn-local\trained_models_02112022\single-task-scar\resnet34d_384_LVEF50\model.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v1.9.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file c:\Users\namea\Desktop\ecg\ecg-cnn-local\trained_models_02112022\single-task-lvef\resnet34d_384_LVEF50\model.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v1.9.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file c:\Users\namea\Desktop\ecg\ecg-cnn-local\trained_models_02112022\1-stage-multitask\resnet34d_384_LVEF50\model.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.6.

In [6]:
csv_path = "../../ECG_EF_Clin_train_dev_new.csv"
image_dir = "../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_images_new/"
df = load_ecg_dataframe(csv_path, image_dir, drop_impute=False, do_split=True)
old_test_df = df[df["split"] == "old_test"].reset_index(drop=True)

df = load_ecg_dataframe(csv_path, image_dir, drop_impute=True, do_split=True)
old_test_df_no_impute = df[df["split"] == "old_test"].reset_index(drop=True)

csv_path = "../../ECG_EF_Clin_test_new_nocut_noimpute.csv"
image_dir = "../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_test_images_new/"
new_test_df = load_ecg_dataframe(csv_path, image_dir, drop_impute=True, do_split=False)

## **Evaluation**

### **single-task**

In [4]:
scar_result_df, scar_metric_df = evaluate_from_dataframe(old_test_df, singletask_scar_classifier)
lvef_result_df, lvef_metric_df = evaluate_from_dataframe(old_test_df, singletask_lvef_classifier)

  0%|          | 0/406 [00:00<?, ?it/s]

  0%|          | 0/406 [00:00<?, ?it/s]

### **multi-task**

In [15]:
old_result_df, old_metric_df = evaluate_from_dataframe(old_test_df, multitask_classifier)
old_no_impute_result_df, old_no_impute_metric_df = evaluate_from_dataframe(old_test_df_no_impute, multitask_classifier)

new_result_df, new_metric_df = evaluate_from_dataframe(new_test_df, multitask_classifier)

  0%|          | 0/1045 [00:00<?, ?it/s]

  0%|          | 0/406 [00:00<?, ?it/s]

  0%|          | 0/1483 [00:00<?, ?it/s]

In [16]:
old_metric_df, old_no_impute_metric_df, new_metric_df

(                 scar      lvef
 Accuracy     0.709091  0.881340
 Sensitivity  0.772563  0.885417
 Specificity  0.686198  0.880422
 F1           0.584699  0.732759
 AUC          0.823789  0.948188
 FPR          0.313802  0.119578
 FNR          0.227437  0.114583,
                  scar      lvef
 Accuracy     0.704433  0.901478
 Sensitivity  0.768519  0.920000
 Specificity  0.681208  0.897281
 F1           0.580420  0.775281
 AUC          0.798378  0.960040
 FPR          0.318792  0.102719
 FNR          0.231481  0.080000,
                  scar      lvef
 Accuracy     0.740391  0.880647
 Sensitivity  0.748130  0.718631
 Specificity  0.737523  0.915574
 F1           0.609137  0.681081
 AUC          0.836965  0.933920
 FPR          0.262477  0.084426
 FNR          0.251870  0.281369)

### **multi-task-clinical**

In [17]:
old_result_df, old_metric_df = evaluate_from_dataframe(old_test_df, multitask_clinical_classifier)
old_no_impute_result_df, old_no_impute_metric_df = evaluate_from_dataframe(old_no_impute_test_df, multitask_clinical_classifier)

new_result_df, new_metric_df = evaluate_from_dataframe(new_test_df, multitask_clinical_classifier)

  0%|          | 0/1045 [00:00<?, ?it/s]

  0%|          | 0/406 [00:00<?, ?it/s]

  0%|          | 0/1483 [00:00<?, ?it/s]

In [18]:
old_metric_df, old_no_impute_metric_df, new_metric_df

(                 scar      lvef
 Accuracy     0.789474  0.875598
 Sensitivity  0.678700  0.901042
 Specificity  0.829427  0.869871
 F1           0.630872  0.726891
 AUC          0.839012  0.945142
 FPR          0.170573  0.130129
 FNR          0.321300  0.098958,
                  scar      lvef
 Accuracy     0.766010  0.884236
 Sensitivity  0.648148  0.880000
 Specificity  0.808725  0.885196
 F1           0.595745  0.737430
 AUC          0.795659  0.951742
 FPR          0.191275  0.114804
 FNR          0.351852  0.120000,
                  scar      lvef
 Accuracy     0.840863  0.877950
 Sensitivity  0.583541  0.741445
 Specificity  0.936229  0.907377
 F1           0.664773  0.683012
 AUC          0.852027  0.926505
 FPR          0.063771  0.092623
 FNR          0.416459  0.258555)